## Pipeline for HiC Analysis and visualization of TADs with significant interactions

Major steps:

* Obtain genome-wide contact matrix by aligning reads to genome using Juicer program 
* Convert juicer aligment output to tag file compatible with Homer
* Extract significant interactions of region(s) of interest using Homer
* Identify tads and loops using homer
* Generate bedpe file with output with significant interaction from Homer
* Convert juicer matrix to .h5 (cool) matrix to plot with HicExplorer
* Generate a track file 
* Plot TAD and interactions using HicExplorer





### Prerequisites 
 * juicer <= 1.6 including the restriction sites files
 * homer
 * hicexplorer <= 3.6

### Input
***Raw sequencing files:***  
 * HIFLR_merged_R1.fastq.gz
 * HIFLR_merged_R2.fastq.gz
 * SRR9016013_R1.fastq
 * SRR9016013_R2.fastq
 
 
See methods for information to access ```HIFLR_merged_R[1,2].fastq.gz``` files from the GEO repository of this publication.

```SRR9016013_R[1,2].fastq``` correspond to ESC (Naive) files downloaded from Kurup J.T., et al., 2020 https://doi.org/10.1242/dev.188516



### Obtain genome-wide contact matrix
Align reads to genome using Juicer program

In [1]:
#adapt to your environment
PWD_HIFLR="path_to_raw_data_folder"
PWD_ESC="path_to_esc_data_folder"
ARIMA='/path/to/mm10_Arima.txt'



[+] Loading juicer  1.6 



In [ ]:
#run the juicer pipeline with the Naive (ESC) raw data and the restriction sites file
cd $PWD_ESC

juicer.sh -d $PWD_ESC -g mm10 -y $ARIMA

In [ ]:
#run the juicer pipeline with the HIFLR raw data and the restriction sites file
cd $PWD_HILFR

juicer.sh -d $PWD_HILFR -g mm10 -y $ARIMA

### Extract significant interactions from regions of interest 
#### Region of interest: Pax7 Intron 7th

Intron 7th (950bp)Coordinates: 139771035-139771985

Steps:

1) Generate HiC-Summary file from merged_nodps.txt from Juicer alignment
2) Generate reference tag files 
3) Identify significant interactions of region of interest 
4) Find TADs and Loops
5) Create a bedpe file from homer significant interactions output (from step 3)

In [ ]:
#step 1

# Generate HiC-summary from alignment ESC Published data set
# Juicer output is in folder PWD_ESC/juicer
# save output summary file from this command in PWD_ESC/homer 
#for downstream analysis with homer

cd $PWD_ESC

awk 'BEGIN {OFS="\t"};{print $15,$2,$3,($1 == "0" ? "+" : "-"),$6,$7,($5 == "0" ? "+" : "-")}' /juicer/aligned/merged_nodups.txt > /homer/ESC_merged_nodups_sum.txt

In [ ]:
# Generate HiC-summary from alignment HIFLR data set
# Juicer output is in folder PWD_HIFLR/juicer
# save output summary file from this command in PWD_HIFLR/homer 
# for downstream analysis with homer

cd $PWD_HIFLR

awk 'BEGIN {OFS="\t"};{print $15,$2,$3,($1 == "0" ? "+" : "-"),$6,$7,($5 == "0" ? "+" : "-")}' /juicer/aligned/merged_nodups.txt > /homer/HIFLR_merged_nodups_sum.txt

In [ ]:
# step 2

# Generate reference tag files from ESC HiC-Summary 

cd $PWD_ESC

makeTagDirectory ESC-HiC -format HiCsummary /homer/ESC_merged_nodups_sum.txt


In [ ]:
# Generate reference tag files from HILFR HiC-Summary 

cd $PWD_HILFR

makeTagDirectory HILFR-HiC -format HiCsummary /homer/HILFR_merged_nodups_sum.txt


In [ ]:
# step 3

#Identify and extract significant interaction of Intron 7th in ESC dataset
# here we tested different resolution and window sizes and
# determined that given this dataset the optimal sizes were set 
# at -res 10k -window 25k

cd $PWD_ESC/homer

analyzeHiC ESC-HiC/ -pos chr4:139771035-139771985 -vsGenome -res 10000 -window 25000 -cpu 28 -interactions ESC_intron7.txt -nomatrix


In [ ]:
#Identify and extract significant interaction of Intron 7th in HLFR dataset
# here we tested different resolution and window sizes and
# determined that given this dataset the optimal sizes were set 
# at -res 10k -window 25k

cd $PWD_HILFR/homer

analyzeHiC HILFR-HiC/ -pos chr4:139771035-139771985 -vsGenome -res 10000 -window 25000 -cpu 28 -interactions HILFR_intron7.txt -nomatrix


In [ ]:
# step 4

# Identify tads and loops in ESC HiC dataset

cd $PWD_ESC/homer
findTADsAndLoops.pl find ESC-HiC/ -cpu 28 -res 10000 -window 25000 -genome mm10 -o ESC_tads_and_loops


In [ ]:
# Identify tads and loops in HILFR HiC dataset

cd $PWD_HILFR/homer
findTADsAndLoops.pl find ESC-HiC/ -cpu 28 -res 10000 -window 25000 -genome mm10 -o HILFR_tads_and_loops


In [ ]:
# step 5

# generate Bedpe file from significant interactions extracted in step 3

## Intron 7 significant interactions in ESC

library(dplyr)

# ESC
esc <- "path_to_ESC_homer_output_folder"
file_dir <- esc
setwd(file_dir)

#ESC -10kb resolution, 25kb window size
hic_files_10x25 <- "ESC_intron7.txt"
offsets_10x25<-c(1000,1000,1000,1000)

#get a list of all genomic region object for the above files 
hic_data <- lapply((1:length(hic_files)), getGenomicInteractions, description= description)

#combine them into a single object
#This is necessary when ploting interactions from 
#multiple regions of interest (For instance Promoter and intron 7 -E7)
hic_data <- do.call(c, hic_data)
as.data.frame(hic_data)


data_esc <-(as.data.frame(hic_data))

## Adding offsets
data_offsets <- data_esc

data_offsets$start1 <- data_offsets$start1 +1000
data_offsets$end1 <- data_offsets$end1 +1000
data_offsets$start2 <- data_offsets$start2 +1000
data_offsets$end2 <- data_offsets$end2 +1000

# select data for bedpe file
toplot <- select(datamb_offsets, c('seqnames1', 'start1', 'end1', 'seqnames2', 'start2', 'end2'))

# Writing bedpe

write.table(toplot, file = "esc_intron7.bedpe", row.names=FALSE, col.names=FALSE, sep="\t", quote=FALSE)




In [ ]:

# Generate Bedpe file from significant interactions extracted in step 3

## Intron 7 significant interactions in HILFR

library(dplyr)

# HILFR
hilfr <- "path_to_HILFR_homer_output_folder"
file_dir <- hilfr
setwd(file_dir)

#ESC -10kb resolution, 25kb window size
hic_files_10x25 <- "HILFR_intron7.txt"
offsets_10x25<-c(1000,1000,1000,1000)

#get a list of all genomic region object for the above files 
hic_data <- lapply((1:length(hic_files)), getGenomicInteractions, description= description)

#combine them into a single object
#This is necessary when ploting interactions from 
#multiple regions of interest (For instance Promoter and intron 7 -E7)
hic_data <- do.call(c, hic_data)
as.data.frame(hic_data)


data_hilfr <-(as.data.frame(hic_data))

## Adding offsets
data_offsets <- data_hilfr

data_offsets$start1 <- data_offsets$start1 +1000
data_offsets$end1 <- data_offsets$end1 +1000
data_offsets$start2 <- data_offsets$start2 +1000
data_offsets$end2 <- data_offsets$end2 +1000

# select data for bedpe file
toplot <- select(datamb_offsets, c('seqnames1', 'start1', 'end1', 'seqnames2', 'start2', 'end2'))

# Writing bedpe

write.table(toplot, file = "hilfr_intron7.bedpe", row.names=FALSE, col.names=FALSE, sep="\t", quote=FALSE)





### Plot TAD and interactions 
#### Region of interest: Pax7 Intron 7th

Intron 7th (950bp)Coordinates: 139771035-139771985

Steps:

1) Convert juicer ```.hic``` to ```.cool``` using hicConvertFormat from HiCExplorer

2) Create a ```_track.ini``` file containing the track information (see example file in next cell)
 
3) Run hicPlotTADs

In [ ]:
# step 1
# Convert juicer .hic to cooler using hicConvertFormat  from HiCExplorer

cd $PWD_ESC/juicer

hicConvertFormat --matrices aligned/inter_30.hic \
                 --outFileName  esc.cool \
                 --inputFormat  hic \
                 --outputFormat cool

cd $PWD_HILFR/juicer

hicConvertFormat --matrices aligned/inter_30.hic \
                 --outFileName  hiflr.cool \
                 --inputFormat  hic \
                 --outputFormat cool

In [ ]:
# step 2

## track file template - this template contains TADS and interaction. 


[x-axis]
where = top

[hic matrix]
file = esc.mcool::/resolutions/10000
title = Naïve
# depth is the maximum distance plotted in bp. In Hi-C tracks
# the height of the track is calculated based on the depth such
# that the matrix does not look deformed
depth = 1000000
transform = log1p
file_type = hic_matrix

[tads]
file = ESC_tad.2D.bed
file_type = domains
border_color = black
color = none
# the tads are overlay over the hic-matrix
# the share-y options sets the y-axis to be shared
# between the Hi-C matrix and the TADs.
overlay_previous = share-y

[spacer]

[hic matrix 2]
file = HILFR.mcool::/resolutions/10000
title = HILFR
# depth is the maximum distance plotted in bp. In Hi-C tracks
# the height of the track is calculated based on the depth such
# that the matrix does not look deformed
depth = 1000000
transform = log1p
file_type = hic_matrix

[tads 2]
file = HILFR.tad.2D.bed
file_type = domains
border_color = black
color = none
# the tads are overlay over the hic-matrix
# the share-y options sets the y-axis to be shared
# between the Hi-C matrix and the TADs.
overlay_previous = share-y

[spacer]

[gene annotation]
file = Path_to_annotation_file: mouse_genome/mm10.refGene.gtf.gz
height = 10
title = mm10 genes
merge_transcripts = true
prefered_name = gene_name
fontsize = 12
file_type = gtf

In [ ]:
# step 3 
#plot TAD at the region of interest with atac seq and interaction tracks

hicPlotTADs --tracks esc_hiflr_track.ini
            -o atac.esc.8r1.6r2.interactions.with.offsets.gray.v4.pdf 
            --region chr4:139,700,000-139,900,000
    